In [7]:
import cv2

# Initialize video capture object from default camera
cap = cv2.VideoCapture('Media1.mp4')

# Define initial frame for background subtraction
ret, frame = cap.read()
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (21, 21), 0)
bg = gray

while True:
    # Read frame from video capture object
    ret, frame = cap.read()

    # Convert frame to grayscale and apply Gaussian blur
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (21, 21), 0)

    # Calculate absolute difference between background and current frame
    diff = cv2.absdiff(bg, gray)

    # Apply threshold to remove small differences and obtain binary image
    thresh = cv2.threshold(diff, 25, 255, cv2.THRESH_BINARY)[1]

    # Dilate binary image to fill in holes and find contours of moving objects
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    thresh = cv2.dilate(thresh, kernel, iterations=4)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw bounding boxes around moving objects and calculate their centroids
    for contour in contours:
        if cv2.contourArea(contour) < 1000:
            continue
        (x, y, w, h) = cv2.boundingRect(contour)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        centroid_x = int(x + w/2)
        centroid_y = int(y + h/2)
        cv2.circle(frame, (centroid_x, centroid_y), 3, (0, 0, 255), -1)

    # Show resulting image with bounding boxes and centroids
    cv2.imshow("Frame", frame)

    # Update background model using current frame
    bg = gray

    # Wait for key press to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture object and close windows
cap.release()
cv2.destroyAllWindows()


error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
